In [1]:
import pandas as pd 
import pickle
import warnings
warnings.filterwarnings('ignore')
import numpy as np

from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score

In [2]:
labels = pd.read_csv("../../data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,BBT,0.0,NONE
1,1_10009,1_10009_u,BBT,0.0,NONE
2,1_1001,1_1001_u,BBT,0.0,NONE
3,1_1003,1_1003_u,BBT,1.0,PRO
4,1_10190,1_10190_u,BBT,0.0,NONE


#### Perform mean, median, max, min and sum pooling on audio feature data

In [4]:
def get_audio_mean_pool(audio) -> np.ndarray:
        return np.array([np.mean(feature_vector, axis=1) for feature_vector in audio])
    
def get_audio_median_pool(audio) -> np.ndarray:
        return np.array([np.median(feature_vector, axis=1) for feature_vector in audio])
    
def get_audio_max_pool(audio) -> np.ndarray:
        return np.array([np.max(feature_vector, axis=1) for feature_vector in audio])

def get_audio_min_pool(audio) -> np.ndarray:
        return np.array([np.min(feature_vector, axis=1) for feature_vector in audio])

def get_audio_sum_pool(audio) -> np.ndarray:
        return np.array([np.sum(feature_vector, axis=1) for feature_vector in audio])

In [5]:
def get_model_data(audio_features):
    model_data = pd.DataFrame(columns=['audio_feature','sarcasm','sarcasm_type'])
    for index, row in labels.iterrows():
        audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'audio_feature': audio_features[audio_key],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"]},
                                  ignore_index=True)
    return model_data

In [6]:
def get_train_test_split(model_data, x_column, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_column],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=0, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    return X_train, X_test, Y_train, Y_test

def get_pooled_data(X_train, X_test, pool_type):
    if pool_type == "mean":
        X_train_mean = get_audio_mean_pool(X_train)
        X_test_mean = get_audio_mean_pool(X_test)
        return X_train_mean, X_test_mean
    if pool_type == "median":
        X_train_median = get_audio_median_pool(X_train)
        X_test_median = get_audio_median_pool(X_test)
        return X_train_median, X_test_median
    if pool_type == "max":
        X_train_max = get_audio_max_pool(X_train)
        X_test_max = get_audio_max_pool(X_test)
        return X_train_max, X_test_max
    if pool_type == "min":
        X_train_min = get_audio_min_pool(X_train)
        X_test_min = get_audio_min_pool(X_test)
        return X_train_min, X_test_min
    if pool_type == "sum":
        X_train_sum = get_audio_sum_pool(X_train)
        X_test_sum = get_audio_sum_pool(X_test)
        return X_train_sum, X_test_sum
    

In [7]:
def svm_classifier(X_train, X_test, Y_train, Y_test):
    svm_clf = svm.SVC(random_state=0, kernel = "rbf", gamma = "scale", class_weight = "balanced")
    C = [0.001, 0.01, 0.1, 1, 10]
    gscv_clf = GridSearchCV(
        estimator=svm_clf, 
        param_grid=dict(C=C),
        n_jobs=-1, 
        cv = 10, 
        scoring = 'f1_macro', 
        refit = True)

    gscv_clf.fit(X_train, Y_train)
    Y_test_pred = gscv_clf.predict(X_test)
    report = classification_report(Y_test, Y_test_pred)
    return report, gscv_clf.best_estimator_

### Librosa

In [8]:
with open('../../audio_features/feat_dict_librosa_lld.pickle', 'rb') as f:
    librosa_audio_features = pickle.load(f, encoding='latin1')
    
model_data = get_model_data(librosa_audio_features)

In [9]:
X_train, X_test, Y_train, Y_test = get_train_test_split(model_data, 'audio_feature', 'sarcasm_type', 'sarcasm_type')
X_train_mean, X_test_mean = get_pooled_data(X_train, X_test, "mean")
X_train_median, X_test_median = get_pooled_data(X_train, X_test, "median")
X_train_max, X_test_max = get_pooled_data(X_train, X_test, "max")
X_train_min, X_test_min = get_pooled_data(X_train, X_test, "min")
X_train_sum, X_test_sum = get_pooled_data(X_train, X_test, "sum")

Train:  (961,) (961,) Test:  ((241,), (241,))


In [10]:
mean_report, mean_best_est = svm_classifier(X_train_mean, X_test_mean, Y_train, Y_test)
median_report, median_best_est = svm_classifier(X_train_median, X_test_median, Y_train, Y_test)
max_report, max_best_est = svm_classifier(X_train_max, X_test_max, Y_train, Y_test)
min_report, min_best_est = svm_classifier(X_train_min, X_test_min, Y_train, Y_test)
sum_report, sum_best_est = svm_classifier(X_train_sum, X_test_sum, Y_train, Y_test)

In [11]:
print("********************************mean report********************************")
print(mean_report)
print(mean_best_est)
print("\n")
print("********************************median report********************************")
print(median_report)
print(median_best_est)
print("\n")
print("********************************max report********************************")
print(max_report)
print(max_best_est)
print("\n")
print("********************************min report********************************")
print(min_report)
print(min_best_est)
print("\n")
print("********************************sum report********************************")
print(sum_report)
print(sum_best_est)

********************************mean report********************************
              precision    recall  f1-score   support

         EMB       0.05      0.12      0.07        17
         ILL       0.20      0.11      0.14        36
         LIK       0.00      0.00      0.00         1
        NONE       0.69      0.53      0.60       120
         PRO       0.34      0.16      0.22        67

    accuracy                           0.34       241
   macro avg       0.26      0.19      0.21       241
weighted avg       0.47      0.34      0.39       241

SVC(C=10, class_weight='balanced', random_state=0)


********************************median report********************************
              precision    recall  f1-score   support

         EMB       0.05      0.12      0.07        17
         ILL       0.11      0.14      0.12        36
         LIK       0.00      0.00      0.00         1
        NONE       0.69      0.39      0.50       120
         PRO       0.41      0.18

### OpenSmile

In [12]:
with open('../../audio_features/feat_dict_opensmile_lld.pickle', 'rb') as f:
    opensmile_audio_features = pickle.load(f, encoding='latin1')
    
model_data = get_model_data(opensmile_audio_features)

In [13]:
X_train, X_test, Y_train, Y_test = get_train_test_split(model_data, 'audio_feature', 'sarcasm_type', 'sarcasm_type')
X_train_mean, X_test_mean = get_pooled_data(X_train, X_test, "mean")
X_train_median, X_test_median = get_pooled_data(X_train, X_test, "median")
X_train_max, X_test_max = get_pooled_data(X_train, X_test, "max")
X_train_min, X_test_min = get_pooled_data(X_train, X_test, "min")
X_train_sum, X_test_sum = get_pooled_data(X_train, X_test, "sum")

Train:  (961,) (961,) Test:  ((241,), (241,))


In [14]:
mean_report, mean_best_est = svm_classifier(X_train_mean, X_test_mean, Y_train, Y_test)
median_report, median_best_est = svm_classifier(X_train_median, X_test_median, Y_train, Y_test)
max_report, max_best_est = svm_classifier(X_train_max, X_test_max, Y_train, Y_test)
min_report, min_best_est = svm_classifier(X_train_min, X_test_min, Y_train, Y_test)
sum_report, sum_best_est = svm_classifier(X_train_sum, X_test_sum, Y_train, Y_test)

In [15]:
print("********************************mean report********************************")
print(mean_report)
print(mean_best_est)
print("\n")
print("********************************median report********************************")
print(median_report)
print(median_best_est)
print("\n")
print("********************************max report********************************")
print(max_report)
print(max_best_est)
print("\n")
print("********************************min report********************************")
print(min_report)
print(min_best_est)
print("\n")
print("********************************sum report********************************")
print(sum_report)
print(sum_best_est)

********************************mean report********************************
              precision    recall  f1-score   support

         EMB       0.02      0.06      0.03        17
         ILL       0.19      0.47      0.27        36
         LIK       0.00      0.00      0.00         1
        NONE       0.57      0.13      0.22       120
         PRO       0.36      0.27      0.31        67

    accuracy                           0.22       241
   macro avg       0.23      0.19      0.16       241
weighted avg       0.41      0.22      0.24       241

SVC(C=10, class_weight='balanced', random_state=0)


********************************median report********************************
              precision    recall  f1-score   support

         EMB       0.07      0.18      0.11        17
         ILL       0.19      0.14      0.16        36
         LIK       0.00      0.00      0.00         1
        NONE       0.69      0.18      0.29       120
         PRO       0.32      0.52